In [52]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import plotnine as gg
from palettable.colorbrewer.sequential import *
from pathlib import Path
import umap.umap_ as umap
from scipy.stats import ttest_ind


In [53]:
top_dir = os.path.dirname(os.getcwd())
outpath = os.path.join(top_dir, "Figures")


## Data Paths

In [54]:
path = Path("../collated/")
#outpath = Path("/Welch_ttest/")
fname = "C-7210-01-CMP-008-gray_collapsed_sampled.csv"
fpath = os.path.join(path, fname)
df = pd.read_csv(fpath)


metadata = [col for col in df.columns if "Metadata" in col]
var = [col for col in df.columns if not "Metadata" in col]
exclude_var = ['Cells_Correlation_Costes_DNA_Mito',
 'Cytoplasm_Correlation_Costes_DNA_Mito',
 'Cytoplasm_Correlation_Costes_DNA_RNA']

variables = [v for v in var if v not in exclude_var]

len(variables)


595

## Welch Ttest

In [4]:
df['Metadata_Target'].replace(np.nan, 'NA', inplace=True)
df['Metadata_Annotation'].replace('active ', 'active', inplace=True)

p = ['NA', 'SECRET']

excluded = ["C1", "C2", "C3", "C4"]

tt = []

for c in df.Metadata_cell_line.unique():
    d = df.query('Metadata_cell_line in @ c and Metadata_compound_name not in @ excluded and Metadata_Target in @p')

    for i in variables:
        test = d.Metadata_compound_name.unique().tolist()
        gp1 = d.query("Metadata_compound_name in @ test[0]")[i].values
        gp2 = d.query("Metadata_compound_name in @ test[1]")[i].values
        gp3 = d.query("Metadata_compound_name in @ test[2]")[i].values
    

        t1 = ttest_ind(gp1, gp2, equal_var=False)
        t2 = ttest_ind(gp1, gp3, equal_var=False)
        t3 = ttest_ind(gp2, gp3, equal_var=False)



        d1 = {"Paired": [test[0]+', '+ test[1], test[0]+', '+ test[2], test[1]+', '+ test[2]],
          "Statistics": [t1[0], t2[0], t3[0]],
          "P-value": [t1[1], t2[1], t3[1]],
          "Variables": str(i),
          "Cellline": str(c)}



        prf = pd.DataFrame(d1, columns = ['Paired','Statistics', 'P-value', 'Variables', "Cellline"])
    
        tt.append(prf)
final_tt = pd.concat(tt)

    
final_tt['logp'] = -log10(final_tt['P-value'])
index = [i for i in range(1, len(final_tt) +1)]
final_tt['index'] =  index
 
    


## Plotting top 15 features which are different between Active and Inactive controls

In [39]:
c = "A549"

tmp = (final_tt.query("Paired == 'BSJ-04-030, BSJ-03-136' and Cellline in @ c")
      .rename(columns={"logp": "Active_Inactive"})
      .reset_index()
      .drop(columns=["level_0", "index"])
      .sort_values(by=["Active_Inactive"],ascending=False)
     )

top_var = tmp.head(15)['Variables'].tolist()
bottom_var = tmp.tail(15)['Variables'].tolist()


## Plotting Density plots for those top and bottom features


p = ["NA", 'SECRET']

excluded = ["C1", "C2", "C3", "C4"]


d = df.query('Metadata_cell_line in @ c and Metadata_compound_name not in @ excluded and Metadata_Target in @p')




for i, var in enumerate(top_var):
    g = gg.ggplot() + \
    gg.geom_density(gg.aes(x=str(var), y='stat(density)', color = 'Metadata_compound_name', fill= 'Metadata_compound_name'), data= d, alpha=.1) + \
    gg.xlab(str(var)) + \
    gg.ylab("Density") + \
    gg.labs(title='Density plot') + \
    gg.theme_classic() 



    gg.ggsave(filename= str(var)+ ".png", plot = g, path = os.path.join(outpath, "Densityplot", c, "top_variables"))


for i, var in enumerate(bottom_var):
    g = gg.ggplot() + \
    gg.geom_density(gg.aes(x=str(var), y='stat(density)', color = 'Metadata_compound_name', fill= 'Metadata_compound_name'), data= d, alpha=.1) + \
    gg.xlab(str(var)) + \
    gg.ylab("Density") + \
    gg.labs(title='Density plot') + \
    gg.theme_classic() 



    gg.ggsave(filename= str(var)+ ".png", plot = g, path = os.path.join(outpath, "Densityplot", c, "bottom_variables"))

###  Part1:  Plotting features selected from 95th percent threshold -log10(Pvalues) of control_Active

In [40]:
d1 = (final_tt.query("Paired == 'DMSO, BSJ-04-030'")
      .rename(columns={"logp": "Control_Inactive"})
      .reset_index()
      .drop(columns=["level_0", "index"])
     )


d2 = (final_tt.query("Paired == 'DMSO, BSJ-03-136'")
      .rename(columns = {"logp": "Control_Active"})
      .reset_index()
      .drop(columns=["level_0", "index", "Cellline", "Variables"])
)



result = pd.concat([d1, d2], axis=1)


col_var = ["Control_Inactive", "Control_Active", "Variables", "Cellline"]

prf = result.loc[:, col_var]



c = "A549"

pf = prf.query("Cellline in @ c")




control_active = pf.Control_Active.quantile(0.95)
control_inactive = pf.Control_Inactive.quantile(0.95)



# # # only label points which are greater than threshold


selected_var = (pf.query("Control_Active > @ control_active")
                .sort_values(by='Control_Active', ascending=False)
               )




# #selected_var = pf.query("Control_Active > @ control_active and Control_Inactive > @ control_inactive")

g = gg.ggplot(pf, gg.aes(x='Control_Inactive', y='Control_Active', label="Variables")) + \
    gg.geom_point(size = 1.5, color="#de2d26") + \
    gg.geom_vline(xintercept= control_inactive, color="blue",linetype='dashed') + \
    gg.theme_classic() + \
    gg.geom_label(data = selected_var,
                 size=8,
                 label_size=0,
                 label_padding=1,
                 show_legend=False
                ) + \
    gg.geom_hline(yintercept = control_active, color="black") + \
    gg.labs(title='SECRET ' + "[" + str(c) + "]" , x="-log10 [p-value] \n\n Control_Inactive", y="Control_Active \n\n -log10 [p-value] ")

    
  
 
gg.ggsave(filename='Welch_test_SECRET_95percentile_labeltext' + str(c) + ".png", plot = g, path = outpath)


g



### Part 2: Plotting density plots

In [55]:
c = "A549"

p = ["NA", 'SECRET']

excluded = ["C1", "C2", "C3", "C4"]

top_var = selected_var['Variables'].tolist()

df['Metadata_Target'].replace(np.nan, 'NA', inplace=True)
df['Metadata_Annotation'].replace('active ', 'active', inplace=True)


d = df.query('Metadata_cell_line in @ c and Metadata_compound_name not in @ excluded and Metadata_Target in @p')




for i, var in enumerate(top_var):
    g = gg.ggplot() + \
    gg.geom_density(gg.aes(x=str(var), y='stat(density)', color = 'Metadata_compound_name', fill= 'Metadata_compound_name'), data= d, alpha=.1) + \
    gg.xlab("Feature_Name") + \
    gg.ylab("Density") + \
    gg.labs(title=str(var)) + \
    gg.theme_classic() 



    gg.ggsave(filename= str(var)+ ".png", plot = g, path = os.path.join(outpath, "Densityplot_new", c, "top_variables"))



## Welch ttest for ERK5

In [ ]:
tt = []

for i in variables:
    test = d.Metadata_compound_name.unique().tolist()
    gp0 = d.query("Metadata_compound_name in @ test[0]")[i].values
    gp1 = d.query("Metadata_compound_name in @ test[1]")[i].values
    gp2 = d.query("Metadata_compound_name in @ test[2]")[i].values
    gp3 = d.query("Metadata_compound_name in @ test[3]")[i].values
    

    t0 = ttest_ind(gp0, gp1, equal_var=False)
    t1 = ttest_ind(gp0, gp2, equal_var=False)
    t2 = ttest_ind(gp0, gp3, equal_var=False)
    t3 = ttest_ind(gp1, gp2, equal_var=False)
    t4 = ttest_ind(gp1, gp3, equal_var=False)
    t5 = ttest_ind(gp2, gp3, equal_var=False)



    d1 = {"Paired": [test[0]+', '+ test[1], test[0]+', '+ test[2], test[0]+', '+ test[3], test[1]+', '+ test[2], test[1]+', '+ test[3], test[2]+', '+ test[3]],
      "Statistics": [t0[0], t1[0], t2[0], t3[0], t4[0], t5[0]],
      "P-value": [t0[1], t1[1], t2[1], t3[1], t4[1], t5[1]],
      "Variables": str(i)}



    prf = pd.DataFrame(d1, columns = ['Paired','Statistics', 'P-value', 'Variables'])
    
    tt.append(prf)
final_tt = pd.concat(tt)
    
final_tt['logp'] = -log10(final_tt['P-value'])
index = [i for i in range(1, len(final_tt) +1)]
final_tt['index'] =  index
final_tt.head()   

In [ ]:
final_tt.to_csv("ERK5_ttest_U2OS.csv",index=False)

### Welch ttest

In [ ]:
tt = []

for i in variables:
    test = d.Metadata_compound_name.unique().tolist()
    gp1 = d.query("Metadata_compound_name in @ test[0]")[i].values
    gp2 = d.query("Metadata_compound_name in @ test[1]")[i].values
    gp3 = d.query("Metadata_compound_name in @ test[2]")[i].values
    

    t1 = ttest_ind(gp1, gp2, equal_var=False)
    t2 = ttest_ind(gp1, gp3, equal_var=False)
    t3 = ttest_ind(gp2, gp3, equal_var=False)



    d1 = {"Paired": [test[0]+', '+ test[1], test[0]+', '+ test[2], test[1]+', '+ test[2]],
      "Statistics": [t1[0], t2[0], t3[0]],
      "P-value": [t1[1], t2[1], t3[1]],
      "Variables": str(i)}



    prf = pd.DataFrame(d1, columns = ['Paired','Statistics', 'P-value', 'Variables'])
    
    tt.append(prf)
final_tt = pd.concat(tt)
    
final_tt['logp'] = -log10(final_tt['P-value'])
index = [i for i in range(1, len(final_tt) +1)]
final_tt['index'] =  index
final_tt.head()  